In [1]:
#!sudo pip3 install "foolbox==2.4.0"

In [2]:
import time
import warnings
import torch
import torchvision
import foolbox
import numpy as np
print(torch.__version__)
print(torchvision.__version__)
print(foolbox.__version__)
print(np.__version__)

1.4.0+cu100
0.5.0+cu100
2.4.0
1.19.0


In [3]:
model = torchvision.models.resnet18(pretrained=True).eval()

In [4]:
preprocessing = (np.asarray([0.485, 0.456, 0.406]).reshape((3, 1, 1)), np.asarray([0.229, 0.224, 0.225]).reshape((3, 1, 1)))
fmodel = foolbox.models.PyTorchModel(model, bounds=(0, 1), num_classes=1000, preprocessing=preprocessing)

In [5]:
images = np.load("images.npy")
labels = np.load("labels.npy")
print(images.shape, labels.shape)

(16, 3, 224, 224) (16,)


In [6]:
print(foolbox.utils.accuracy(fmodel, images, labels))

0.9375


In [7]:
%timeit foolbox.utils.accuracy(fmodel, images, labels)

12 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit foolbox.utils.accuracy(fmodel, images[:1], labels[:1])

4.99 ms ± 378 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
attack = foolbox.attacks.PGD(fmodel, distance=foolbox.distances.Linf)

In [10]:
def run_attack(attack, fmodel, images, labels, epsilons):
    advs = attack(images, labels, unpack=False)
    success = [[adv.distance.value <= eps for eps in epsilons] for adv in advs]
    success = np.asarray(success).T
    return success

In [11]:
# apply the attack
start = time.time()
attack = foolbox.attacks.PGD(fmodel, distance=foolbox.distances.Linf)
epsilons = [0.002]
success = run_attack(attack, fmodel, images, labels, epsilons=epsilons)
print(f"attack took {time.time() - start:.1f} seconds")

attack took 37.5 seconds


In [12]:
# calculate and report the robust accuracy
robust_accuracy = 1 - success.mean(axis=-1)
for eps, acc in zip(epsilons, robust_accuracy):
    print(eps, acc.item())

0.002 0.0625


In [13]:
# apply the attack
start = time.time()
attack = foolbox.attacks.PGD(fmodel, distance=foolbox.distances.Linf)
epsilons = [0.0, 0.001, 0.01, 0.03, 0.1, 0.3, 0.5, 1.0]
success = run_attack(attack, fmodel, images, labels, epsilons=epsilons)
print(f"attack took {time.time() - start:.1f} seconds")

attack took 36.9 seconds


In [14]:
# calculate and report the robust accuracy
robust_accuracy = 1 - success.mean(axis=-1)
for eps, acc in zip(epsilons, robust_accuracy):
    print(eps, acc.item())

0.0 0.9375
0.001 0.375
0.01 0.0
0.03 0.0
0.1 0.0
0.3 0.0
0.5 0.0
1.0 0.0
